In [1]:
!pip -qqq install 'crewai[tools]'
!pip -qqq install youtube-transcript-api
!pip -qqq install yt_dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 600.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.7/462.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from crewai import Agent
from google.colab import userdata
from crewai import Crew, Process
from crewai_tools import YoutubeChannelSearchTool
from crewai import Task

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [3]:
OPENAI_API_KEY = userdata.get('OPEN_AI_KEY')
model_ID = userdata.get('GPT_MODEL')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_MODEL_NAME"] = model_ID

In [4]:
# The tool used by the topic researcher
youtube_tool = YoutubeChannelSearchTool(youtube_channel_handle='@techwithzoum')

Processing videos:  18%|█▊        | 12/66 [00:07<00:21,  2.54it/s]ERROR:root:Failed to fetch transcript for video https://www.youtube.com/watch?v=ngTCMKs4070
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/embedchain/loaders/youtube_video.py", line 29, in load_data
    transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=languages)
  File "/usr/local/lib/python3.10/dist-packages/youtube_transcript_api/_api.py", line 137, in get_transcript
    return cls.list_transcripts(video_id, proxies, cookies).find_transcript(languages).fetch(preserve_formatting=preserve_formatting)
  File "/usr/local/lib/python3.10/dist-packages/youtube_transcript_api/_transcripts.py", line 292, in fetch
    return _TranscriptParser(preserve_formatting=preserve_formatting).parse(
  File "/usr/local/lib/python3.10/dist-packages/youtube_transcript_api/_transcripts.py", line 358, in parse
    for xml_element in ElementTree.fromstring(plain_data)
  File "/usr/lib/pyth

1. Agent 1 — Topic Researcher

In [5]:

topic_researcher = Agent(
    role='Topic Researcher',
    goal='Search for relevant videos on the topic {topic} from the provided YouTube channel',
    verbose=True,
    memory=True,
    backstory='Expert in finding and analyzing relevant content from YouTube channels, specializing in AI, Data Science, Machine Learning, and Generative AI topics.',
    tools=[youtube_tool],
    allow_delegation=True
)

2. Agent 2 — Blog Writer

In [6]:
blog_writer = Agent(
    role='Blog Writer',
    goal='Write a comprehensive blog post from the transcription provided by the Topic Researcher, covering all necessary sections',
    verbose=True,
    memory=True,
    backstory='Experienced in creating in-depth, well-structured blog posts that explain technical concepts clearly and engage readers from introduction to conclusion.',
    allow_delegation=False
)

3. Agent 3 — LinkedIn Post Creator

In [7]:
# LinkedIn Post Agent
linkedin_post_agent = Agent(
    role='LinkedIn Post Creator',
    goal='Create a concise LinkedIn post summary from the transcription provided by the Topic Researcher.',
    verbose=True,
    memory=True,
    backstory='Expert in crafting engaging LinkedIn posts that summarize complex topics and include trending hashtags for maximum visibility.',
    allow_delegation=False
)

4. Agent 4 — Twitter Post Creator

In [8]:
twitter_agent = Agent(
    role='Twitter Content Creator',
    goal='Create a short tweet from the transcription provided by the Topic Researcher that capture key points and insights',
    verbose=True,
    memory=True,
    backstory='Specializes in distilling complex information into concise, impactful tweets that resonate with a tech-savvy audience.',
    allow_delegation=False
)

In [9]:
research_task = Task(
    description="Identify and analyze videos on the topic {topic} from the specified YouTube channel.",
    expected_output="A complete word by word report on the most relevant video found on the topic {topic}.",
    agent=topic_researcher,
    tools=[youtube_tool]
)


In [10]:
blog_writing_task = Task(
    description=""" Write a comprehensive blog post based on the transcription provided by the Topic Researcher.
                    The article must include an introduction , step-by-step guides, and conclusion.
                    The overall content must be about 1200 words long.""",
    expected_output="A markdown-formatted of the blog",
    agent=blog_writer,
    output_file='./data/blog-post.md'
)

In [11]:
linkedin_post_task = Task(
    description="Create a LinkedIn post summarizing the key points from the transcription provided by the Topic Researcher, including relevant hashtags.",
    expected_output="A markdown-formatted of the LinkedIn post",
    agent=linkedin_post_agent,
    output_file='./data/linkedin-post.md'
)

In [12]:
twitter_task = Task(
    description="Create a tweet from the transcription provided by the Topic Researcher, including relevant hastags.",
    expected_output="A markdown-formatted of the Twitter post",
    agent=twitter_agent,
    output_file='./data/tweets.md'
)

In [13]:
my_crew = Crew(
    agents=[topic_researcher, linkedin_post_agent, twitter_agent, blog_writer],
    tasks=[research_task, linkedin_post_task, twitter_task, blog_writing_task],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    max_rpm=100,
    share_crew=True
)

In [14]:
topic_of_interest = 'GPT3.5 Turbo Fine-tuning and Graphical Interface'
result = my_crew.kickoff(inputs={'topic': topic_of_interest})


 [2024-09-04 17:56:58][DEBUG]: == Working Agent: Topic Researcher
 [2024-09-04 17:56:58][INFO]: == Starting Task: Identify and analyze videos on the topic GPT3.5 Turbo Fine-tuning and Graphical Interface from the specified YouTube channel.


> Entering new CrewAgentExecutor chain...


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.84it/s]


I need to search for relevant videos on the topic of GPT-3.5 Turbo Fine-tuning and Graphical Interface from the specified YouTube channel (@techwithzoum).

Action: Search a Youtube Channels content
Action Input: {"search_query": "GPT-3.5 Turbo Fine-tuning and Graphical Interface"} 

Relevant Content:
GPT 3.5 turbo model was made available for fine tuning a few days ago and in this tutorial I'll explain how you can create a personal assistant using your custom data the data set I will be using is the Yahoo question answering data sets it has two main columns the question column and the answer column let's get back to the notebook and the first thing to do is to install the data sets package which is used to download data set from hugging face as you can see here the data set is available on hugging phase after this package is installed we import from data set we'll load data set function we provide the name of a data set we're interested in and only taking the training set we can check 